In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
main_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
sample_submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
main_df.head()


In [ ]:
test_df.head()


In [ ]:
#data cleaning
def data_cleaning(df):
    df['comment_text'] = df['comment_text'].replace('\n',' ', regex=True).replace('\r',' ', regex=True).replace('\t',' ', regex=True) #spaces
    df['comment_text'] = df['comment_text'].replace('\\n',' ', regex=True)
    df['comment_text'] = df['comment_text'].replace('http[S|s]*\S+|www.\S+',' ', regex=True) #Links
    df['comment_text'] = df['comment_text'].replace('\d+',' ', regex=True) #numbers
    df['comment_text'] = df['comment_text'].replace('\s{2,}',' ', regex=True) #multiple spaces
    df['comment_text'] = df['comment_text'].map(lambda x: x.strip()) #removing trailSpaces
    df['comment_text'] = df['comment_text'].str.replace('\W', ' ') #special characters => most of the puntuat
    return df


In [ ]:
main_df = data_cleaning(main_df)
test_df = data_cleaning(test_df)


In [ ]:
#tokenization
#def tokenizer(text):
#    return word_tokenize(text)
#print(stopwords.words('english'))
new_stopwords = stopwords.words('english')

def stopwords_remover(words_list):
    text_list = [word for word in words_list if word not in new_stopwords]
    return text_list

lemmitize_obj = nltk.WordNetLemmatizer()

def _lemmatizer(tokenized_text):
    text = [lemmitize_obj.lemmatize(word) for word in tokenized_text]
    return text


In [ ]:
main_df['comment_text_tokenized'] = main_df['comment_text'].apply(lambda x: word_tokenize(x.lower()))
main_df['comment_text_stopword_removed'] = main_df['comment_text_tokenized'].apply(lambda x: stopwords_remover(x))
main_df['comment_text_lemmatized'] = main_df['comment_text_stopword_removed'].apply(lambda x: _lemmatizer(x))

test_df['comment_text_tokenized'] = test_df['comment_text'].apply(lambda x: word_tokenize(x.lower()))
test_df['comment_text_stopword_removed'] = test_df['comment_text_tokenized'].apply(lambda x: stopwords_remover(x))
test_df['comment_text_lemmatized'] = test_df['comment_text_stopword_removed'].apply(lambda x: _lemmatizer(x))

wrk_df = main_df.copy()
wrk_df =  wrk_df.drop(['comment_text','comment_text_stopword_removed','comment_text_tokenized'],axis = 1)
wrk_df['comment_text_lemmatized']= wrk_df['comment_text_lemmatized'].map(lambda x: " ".join(x) ) 
test_df['comment_text_lemmatized']= test_df['comment_text_lemmatized'].map(lambda x: " ".join(x) )


In [ ]:
tf_vect = TfidfVectorizer(lowercase = True, max_features=2000 )


In [ ]:
train_features_sparse = tf_vect.fit_transform(wrk_df['comment_text_lemmatized'])
test_features_sparse = tf_vect.transform(test_df['comment_text_lemmatized'])


In [ ]:
test_df['comment_text_lemmatized'].head()


In [ ]:
clf = ClassifierChain(MultinomialNB(fit_prior=True)).fit(train_features_sparse, wrk_df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']])


In [ ]:
y_predict = clf.predict(test_features_sparse)


In [ ]:
y_predict_proba = clf.predict_proba(test_features_sparse)


In [ ]:
predict_dense = y_predict_proba.todense()
print(predict_dense)


In [ ]:
data_df = pd.DataFrame(predict_dense,columns= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

submission_df_1 = pd.concat([test_df['id'], data_df] ,axis = 1)


In [ ]:
submission_df_1.head()


In [ ]:
submission_df_1.to_csv('submission.csv',index=False)
